In [1]:
from IPython.display import clear_output

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np

import json
import time
import mouse
import os
import sys


In [5]:
def bring_me_YSCaptcha():
    '''
    Вызывает вылет капчи
    '''
    while not driver.find_elements(by='id', value='js-button'):
        url = "https://market.yandex.ru/catalog--igrushki-dlia-detei-do-3-let/59693"
        driver.get(url)

        
def handle_if_YSCaptcha(url:str,
                        x1:int=1021, 
                        x2:int=1284, 
                        y1:int=534, 
                        y2:int=575):
    '''
    Автоматизация для обхода капчи (Параметры функции -- координаты 
    прямоугольника, ограничивающего рабочую область кнопки для прохождения капчи)
    Использовать mouse.get_position() для подбора координат
    '''
    counter = 0
    while driver.find_elements(by='id', value='js-button'):
        print('Работаем с капчей')
        if counter:
            driver.get(url)        
        x = np.random.randint(x1, x2)
        y = np.random.randint(y1, y2)
        mouse.move(x, y, absolute=True, duration=0.1)
        mouse.click('left')
        time.sleep(2.5)
        counter += 1
        

def check_if_loaded(url:str, lag_range:int=3):
    '''
    Декоратор для проверки страницы на ошибки при загрузке
    '''
    def decorator(func):
        def wrapper(*args, **kwargs):
            counter = 0
            for reload in range(2):
                if not counter:
                    pass
                elif not counter % lag_range:
                    driver.get(url)
                    handle_if_YSCaptcha(url)
                else:
                    break

                for lag in range(lag_range):
                    counter += 1
                    time.sleep(lag)
                    tree_lvl3, lvl3_href = func(*args, **kwargs)
                    if tree_lvl3:
                        break

            return tree_lvl3, lvl3_href
        return wrapper
    return decorator

@check_if_loaded(url='url')
def parse_elements_from_page():
    '''
    Функция для сбора данных со страницы
    '''
    check_xpaths = '/html/body/main/div/div[2]/div/div[1]/div[2]/div/div/div/div/div/div/div/div/div/h1'
    categ_xpath_case2 = '/html/body/div[2]/div[2]/div/div/div[2]/div/div[*]/div/div/div/div[1]/div/div/div/div/div/div/div[*]/div[1]/a'
    categ_xpath_case3 = '/html/body/div[2]/div[2]/div/div/div[2]/div/div[*]/div/div/div/div[1]/div/div/div/div/div/div/div[*]/div/a'

    if driver.find_elements(by='xpath', value=check_xpaths):
        category_cards = driver.find_elements(by='xpath', value='//*[@class="_1SFPj"]/li[*]/div/a')
        tree_lvl3 = [category.text for category in category_cards]
        lvl3_href = [card.get_attribute('href') for card in category_cards]

    elif driver.find_elements(by='xpath', value=categ_xpath_case2):
        category_cards = driver.find_elements(by='xpath', value=categ_xpath_case2)
        tree_lvl3 = [category.text for category in category_cards]
        lvl3_href = [card.get_attribute('href') for card in category_cards]

    elif driver.find_elements(by='xpath', value=categ_xpath_case3):
        category_cards = driver.find_elements(by='xpath', value=categ_xpath_case3)
        tree_lvl3 = [category.text for category in category_cards]
        lvl3_href = [card.get_attribute('href') for card in category_cards]
        
    else:
        print('На странице нет искомой информации')
        tree_lvl3, lvl3_href = [], []

    return tree_lvl3, lvl3_href


### Структуру первых трех уровней по html с базовой страницы ЯндексМаркета
https://market.yandex.ru/

In [2]:
# Парсим jsonы из html с категориями (0-2 уровни)
with open('yandex_categories.html', encoding = 'utf-8') as file:
    txt = file.read()

soup = BeautifulSoup(txt, 'lxml')
    
categories_lvl0 = [el.text for el in soup.find_all('span', class_ = '_3krX4')]
json_dict = soup.find_all('noframes', class_ = 'apiary-patch')

level_0, level_1, level_2, lvl2_href = [], [], [], []
parent_url = 'https://market.yandex.ru/'

for json_el in json_dict:
    try:
        tree_lvl0 = list(json.loads(json_el.text)['widgets']['@MarketNode/NavigationTree'].values())[0]['data'][0]
    except Exception as ex:
        continue
    
    tree_lvl0_name = tree_lvl0['fullName']
    tree_lvl1_list = tree_lvl0['navnodes']
    for tree_lvl1 in tree_lvl1_list:
        tree_lvl1_name = tree_lvl1['fullName']
        try:
            tree_lvl2_list = tree_lvl1['navnodes']
        except Exception as ex:
            level_0.append(tree_lvl0_name)
            level_1.append(tree_lvl1_name)
            level_2.append(tree_lvl1_name)
            lvl2_href.append(f'{parent_url}/catalog--{tree_lvl1["slug"]}/{tree_lvl1["link"]["params"]["nid"][0]}\n')
            continue

        for tree_lvl2 in tree_lvl2_list:
            tree_lvl2_name = tree_lvl2['fullName']
            level_0.append(tree_lvl0_name)
            level_1.append(tree_lvl1_name)
            level_2.append(tree_lvl2_name)
            lvl2_href.append(f'{parent_url}catalog--{tree_lvl2["slug"]}/{tree_lvl2["link"]["params"]["nid"][0]}\n')
#             print(tree_lvl0_name, tree_lvl1_name, tree_lvl2_name, sep = '->')
#             print(tree_lvl2['slug'])

result = pd.DataFrame(data = {'level_0': level_0, 
                              'level_1': level_1, 
                              'level_2': level_2, 
                              'lvl2_href': lvl2_href})
result

,level_0,level_1,level_2,lvl2_href
0,Товары со скидкой,Электроника,Смартфоны,https://market.yandex.ru/catalog--smartfony/61...
1,Товары со скидкой,Электроника,Умные часы и браслеты,https://market.yandex.ru/catalog--umnye-chasy-...
2,Товары со скидкой,Электроника,Наушники и Bluetooth-гарнитуры,https://market.yandex.ru/catalog--naushniki-i-...
3,Товары со скидкой,Электроника,Портативная акустика,https://market.yandex.ru/catalog--portativnaia...
4,Товары со скидкой,Электроника,Аудиотехника,https://market.yandex.ru/catalog--audiotekhnik...
...,...,...,...,...
3317,Товары с iHerb,Энергия,Энергия,https://market.yandex.ru//catalog--energiia/50...
3318,Товары с iHerb,Снижение веса,Снижение веса,https://market.yandex.ru//catalog--snizhenie-v...
3319,Товары с iHerb,Минералы,Минералы,https://market.yandex.ru//catalog--mineraly/50...
3320,Товары с iHerb,Моновитамины,Моновитамины,https://market.yandex.ru//catalog--monovitamin...


### Парсим последний уровень категоризатора по ссылкам, собранным на предыдущем этапе

In [8]:
# Загружаем данные
tbl = pd.read_excel('./data/YM_categories.xlsx')
# Фильтруем ненужные категории
remove_lst = ['Товары со скидкой',
              'Продукты',
              'Товары ИКЕА',
              'Уценка',
              'Умные колонки',
              'Товары с iHerb']
tbl = tbl[~tbl['level_0'].isin(remove_lst)]
# Формируем список из URL для обхода парсером
urls = tbl['lvl2_href'].str.strip().unique().tolist()

In [8]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

In [9]:
# Костыльный счетчик для страниц
if os.path.exists('counter.txt'):
    with open('counter.txt') as f:
        i = int(f.readline())
else:
    i = 0

In [ ]:
# Парсер
for url in urls[i:]:
    # Загружаем страницу, проверяем капчу
    driver.get(url)
    handle_if_YSCaptcha(url)
    
    # Переопределяем функцию для текущей страницы
    @check_if_loaded(url)
    def parse_elements_from_page():
        '''
        Функция для сбора данных со страницы
        '''
        # Пример страницы: https://market.yandex.ru/catalog--geiming/26911310/list?rs=eJwdUD1rAkEUvDNVyC8QETatjWgXPWGxCtiovXCVlYW9RRYTJCBY2dmstRG0sFFD1kL_wGEhFi5-YGmKiNjlZpphmHlv5u2mmw9Fd-k6opoL0ZSzIcqTB2U_AwaZEG2Hyi9cO4Qu38DFArrzCVSxOZQit85Q_BG4moLbBN0ulTW4KgBNCb3imfkfcM0KXE6I9ZcQ9Z18DNR_vKHxA36EK1LUK0wYMPkVmf6Ntz2Cy3eg9uBqF9xJ8s420vwWJs2VdwbM4Rt1j738AfVFfcOZLbbMgVuCevwbM3nOR9j4xPY-56P8nx3areKW5G9c0GJr2X_EO4ie&allowCollapsing=1&local-offers-first=0
        category_cards_case1 = driver.find_elements(by=By.CLASS_NAME, value='_1SFPj')
        # Пример страницы: https://market.yandex.ru/catalog--komplektuiushchie-dlia-kompiutera/26912630
        category_cards_case2 = driver.find_elements(by=By.CLASS_NAME, value='_1FXJE')
        
        if category_cards_case1:
            category_cards = category_cards_case1[0].find_elements(by=By.TAG_NAME, value='a')
            tree_lvl3 = [category.text for category in category_cards]
            lvl3_href = [card.get_attribute('href') for card in category_cards]

        elif category_cards_case2:
            category_cards = [category_card.find_element(by=By.TAG_NAME, value='a') \
                                 for category_card in category_cards_case2]
            tree_lvl3 = [category.text for category in category_cards]
            lvl3_href = [card.get_attribute('href') for card in category_cards]

        else:
            print('На странице нет искомой информации')
            tree_lvl3, lvl3_href = [], []

        return tree_lvl3, lvl3_href
        
    # Парсим данные со страницы
    tree_lvl3, lvl3_href = parse_elements_from_page()
    
    # Аппендим данные в csv файл
    print(tree_lvl3)
    pd.DataFrame(data = {'lvl2_href': [url]*len(tree_lvl3),
                         'level_3': tree_lvl3,
                         'lvl3_href': lvl3_href}).to_csv('YM_categories_ext2.csv', 
                                                         mode='a', 
                                                         sep='\t', 
                                                         index=False,
                                                         header = False)

    # Обновляем файл с счетчиком спарсенных строк
    i += 1
    with open('counter.txt', 'w') as f:
        f.write(str(i))
    time.sleep(1)
    clear_output()

else:
    os.remove('counter.txt')